In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# ============================================================
# Golden Cross Scanner & Backtester (Multi-Asset, No Frameworks)
# ============================================================
# Dependencies (install if needed):
#   pip install pandas numpy yfinance matplotlib
#
# README:
# - This script scans for Golden Cross signals (50SMA crossing above 200SMA),
#   creates a signals CSV, then backtests entries on next-day open with
#   multiple exit rules (death cross, ATR trailing stop, hard stop-loss,
#   and max-hold days).
# - Uses only pandas, numpy, yfinance, matplotlib (optional for plots).
# - Entry/exit execution prices use UNADJUSTED OHLC (Open/Close). For daily
#   equity valuation, we use Adjusted Close to passively incorporate corporate
#   actions/dividends. Trade-level P&L does NOT explicitly add dividends.
# - Example usage: set TICKERS and date range below and run.
# - Expected runtime: depends on network/number of tickers; ~2–5 minutes for ~50 tickers.
#
# Outputs:
#   - goldencross_signals.csv
#   - goldencross_backtest_results.csv
#   - equity_curve.csv
#   - equity_curve.png
#   - trade_returns_hist.png
# ============================================================

from __future__ import annotations

import os
import math
import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# ---------------------------
# CONFIGURABLE PARAMETERS
# ---------------------------

# Universe of symbols
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']


# Backtest window
START_DATE = "2020-01-01"
END_DATE   = "2025-09-09"

# Indicators / Signal
FAST_WINDOW = 50         # Fast SMA
SLOW_WINDOW = 200        # Slow SMA
TIMEFRAME   = "1d"       # yfinance interval (e.g., '1d')

# Capital & Sizing
INITIAL_CAPITAL     = 100000.0
POSITION_SIZE_TYPE  = "fixed_percent"   # 'fixed_percent' or 'fixed_amount'
POSITION_PERCENT    = 0.02              # 2% of current capital per new trade if 'fixed_percent'
POSITION_AMOUNT     = 2000.0            # Used if POSITION_SIZE_TYPE == 'fixed_amount'

# Risk & Exits
MAX_HOLD_DAYS             = 30          # Exit at close on this many days after entry
USE_TRAILING_STOP         = True
TRAILING_ATR_MULTIPLIER   = 3.0
ATR_WINDOW                = 14
USE_HARD_STOP             = True
HARD_STOP_PCT             = 0.10         # 10% stop below entry (for longs)

# Frictions
SLIPPAGE_PCT          = 0.0005           # 5 bps slippage per entry/exit (long-only)
COMMISSION_PER_TRADE  = 0.0              # Flat fee per trade

# Outputs
OUTPUT_SIGNALS_CSV       = "output/goldencross_signals.csv"
OUTPUT_BACKTEST_CSV      = "output/goldencross_backtest_results.csv"
OUTPUT_EQUITY_CURVE_CSV  = "output/equity_curve.csv"
os.makedirs("output", exist_ok=True)
# Matplotlib backend save (no GUI needed)
plt.switch_backend("Agg")

# ---------------------------
# Logging setup
# ---------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S",
)
logger = logging.getLogger("golden_cross_backtest")

# ---------------------------
# Data Structures
# ---------------------------

@dataclass
class Signal:
    ticker: str
    signal_date: pd.Timestamp
    entry_date: Optional[pd.Timestamp]
    entry_price: Optional[float]
    fast_sma: float
    slow_sma: float
    atr: Optional[float]
    notes: str = ""


@dataclass
class Position:
    ticker: str
    entry_date: pd.Timestamp
    entry_price_exec: float
    raw_entry_price: float
    shares: int
    highest_close_since_entry: float
    trailing_stop: Optional[float]
    hard_stop: Optional[float]
    max_hold_exit_date: Optional[pd.Timestamp]
    scheduled_exit_next_open: Optional[Tuple[pd.Timestamp, str]] = None  # (date, reason)
    scheduled_exit_at_close: Optional[Tuple[pd.Timestamp, str]] = None   # (date, reason)


@dataclass
class TradeResult:
    ticker: str
    entry_date: pd.Timestamp
    entry_price: float
    exit_date: pd.Timestamp
    exit_price: float
    shares: int
    pnl: float
    return_pct: float
    days_held: int
    reason: str


# ---------------------------
# Utility Functions
# ---------------------------

def to_datetime_index(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure index is tz-naive DatetimeIndex (date-level)."""
    idx = pd.to_datetime(df.index)
    if idx.tz is not None:
        idx = idx.tz_localize(None)
    df = df.copy()
    df.index = idx
    return df


def fetch_data(ticker: str, start: str, end: str, interval: str = "1d") -> Optional[pd.DataFrame]:
    """
    Fetch OHLCV data with yfinance.
    - Unadjusted OHLCV (auto_adjust=False), includes 'Adj Close'.
    - Returns None on failure.
    """
    try:
        df = yf.download(
            tickers=ticker,
            start=start,
            end=end,
            interval=interval,
            auto_adjust=False,
            progress=False,
            threads=True,
            multi_level_index=False
        )
        if df is None or df.empty:
            logger.warning(f"{ticker}: No data returned.")
            return None
        df = to_datetime_index(df)
        # Ensure expected columns exist
        expected = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]
        missing = [c for c in expected if c not in df.columns]
        if missing:
            logger.warning(f"{ticker}: Missing columns {missing}.")
            for c in missing:
                df[c] = np.nan
        df = df[expected].dropna(how="all")
        return df
    except Exception as e:
        logger.error(f"Failed to fetch {ticker}: {e}")
        return None


def compute_indicators(df: pd.DataFrame, fast: int, slow: int, atr_window: int) -> pd.DataFrame:
    """
    Compute Fast/Slow SMAs on Close and ATR (simple rolling) on OHLC.
    """
    df = df.copy()
    df["FAST_SMA"] = df["Close"].rolling(window=fast, min_periods=fast).mean()
    df["SLOW_SMA"] = df["Close"].rolling(window=slow, min_periods=slow).mean()

    # True Range components
    prev_close = df["Close"].shift(1)
    tr1 = df["High"] - df["Low"]
    tr2 = (df["High"] - prev_close).abs()
    tr3 = (df["Low"] - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # ATR as simple moving average of TR
    df["ATR"] = tr.rolling(window=atr_window, min_periods=atr_window).mean()

    return df


def detect_golden_crosses(df: pd.DataFrame, ticker: str) -> List[Signal]:
    """
    Detect Golden Cross: FAST_SMA(t) > SLOW_SMA(t) and FAST_SMA(t-1) <= SLOW_SMA(t-1).
    Entry is at NEXT trading day's open (fallback to next close).
    """
    signals: List[Signal] = []

    cond_today = df["FAST_SMA"] > df["SLOW_SMA"]
    cond_yday = df["FAST_SMA"].shift(1) <= df["SLOW_SMA"].shift(1)
    cross_idx = df.index[cond_today & cond_yday]

    for t in cross_idx:
        # Ensure both SMAs valid at t
        if pd.isna(df.at[t, "FAST_SMA"]) or pd.isna(df.at[t, "SLOW_SMA"]):
            continue

        # Determine next trading day (entry day)
        all_dates = df.index
        loc = all_dates.get_loc(t)
        if isinstance(loc, slice):
            # In rare edge cases; skip
            continue
        next_pos = loc + 1
        if next_pos >= len(all_dates):
            # No next day to enter
            notes = "No next day available for entry"
            signals.append(Signal(
                ticker=ticker,
                signal_date=t,
                entry_date=None,
                entry_price=None,
                fast_sma=float(df.at[t, "FAST_SMA"]),
                slow_sma=float(df.at[t, "SLOW_SMA"]),
                atr=float(df.at[t, "ATR"]) if not pd.isna(df.at[t, "ATR"]) else None,
                notes=notes
            ))
            continue

        entry_day = all_dates[next_pos]
        open_price = df.at[entry_day, "Open"]
        notes = "Entry at next open"
        entry_price = None

        if pd.isna(open_price) or open_price <= 0:
            # Fallback to next close
            close_price = df.at[entry_day, "Close"]
            if pd.isna(close_price) or close_price <= 0:
                notes = "Next open/close invalid"
                entry_day = None
                entry_price = None
            else:
                entry_price = float(close_price)
                notes = "Next open missing -> entry at next close"
        else:
            entry_price = float(open_price)

        signals.append(Signal(
            ticker=ticker,
            signal_date=t,
            entry_date=entry_day,
            entry_price=entry_price,
            fast_sma=float(df.at[t, "FAST_SMA"]),
            slow_sma=float(df.at[t, "SLOW_SMA"]),
            atr=float(df.at[t, "ATR"]) if not pd.isna(df.at[t, "ATR"]) else None,
            notes=notes
        ))

    return signals


def detect_death_cross_dates(df: pd.DataFrame) -> pd.DatetimeIndex:
    """
    Death Cross for exit trigger: FAST_SMA crosses BELOW SLOW_SMA.
    Defined as FAST_SMA(t) < SLOW_SMA(t) and FAST_SMA(t-1) >= SLOW_SMA(t-1).
    """
    cond_today = df["FAST_SMA"] < df["SLOW_SMA"]
    cond_yday = df["FAST_SMA"].shift(1) >= df["SLOW_SMA"].shift(1)
    return df.index[cond_today & cond_yday]


def save_signals_csv(signals: List[Signal], out_path: str):
    rows = []
    for s in signals:
        rows.append({
            "ticker": s.ticker,
            "signal_date": s.signal_date.strftime("%Y-%m-%d") if pd.notna(s.signal_date) else "",
            "entry_date": s.entry_date.strftime("%Y-%m-%d") if s.entry_date is not None else "",
            "entry_price": round(s.entry_price, 6) if s.entry_price is not None else "",
            "fast_sma": round(s.fast_sma, 6) if s.fast_sma is not None else "",
            "slow_sma": round(s.slow_sma, 6) if s.slow_sma is not None else "",
            "atr": round(s.atr, 6) if s.atr is not None else "",
            "notes": s.notes or "",
        })
    df = pd.DataFrame(rows)
    df.to_csv(out_path, index=False)
    logger.info(f"Saved signals to {out_path} ({len(df)} rows).")


def apply_slippage(price: float, is_buy: bool, slippage_pct: float) -> float:
    """
    Adjust price for slippage. For buys: price*(1+slippage), sells: price*(1-slippage)
    """
    if pd.isna(price) or price <= 0:
        return np.nan
    if slippage_pct is None or slippage_pct == 0:
        return float(price)
    if is_buy:
        return float(price) * (1.0 + slippage_pct)
    else:
        return float(price) * (1.0 - slippage_pct)


def first_next_trading_day(df: pd.DataFrame, day: pd.Timestamp) -> Optional[p.Timestamp]:
    """Return the next trading day in df after 'day'."""
    if day not in df.index:
        # find insertion point
        idx = df.index.searchsorted(day)
    else:
        idx = df.index.get_loc(day)
        if isinstance(idx, slice):
            idx = idx.start
        idx += 1
    if idx >= len(df.index):
        return None
    return df.index[idx]


def calculate_metrics(trades_df: pd.DataFrame, equity_curve_df: pd.DataFrame) -> Dict[str, float]:
    """
    Compute performance metrics:
      - Total return & CAGR
      - Annualized volatility
      - Sharpe (rf=0)
      - Max drawdown
      - Win rate, Avg win/loss, # trades
    """
    metrics = {}

    # Total return
    start_val = equity_curve_df["portfolio_value"].iloc[0]
    end_val = equity_curve_df["portfolio_value"].iloc[-1]
    total_return = (end_val / start_val) - 1.0
    metrics["total_return"] = total_return

    # CAGR
    start_date = equity_curve_df["date"].iloc[0]
    end_date = equity_curve_df["date"].iloc[-1]
    days = (end_date - start_date).days
    years = days / 365.25 if days > 0 else 1e-9
    cagr = (end_val / start_val) ** (1.0 / years) - 1.0 if end_val > 0 and start_val > 0 else np.nan
    metrics["cagr"] = cagr

    # Daily returns for equity curve
    eq = equity_curve_df.set_index("date")["portfolio_value"].astype(float)
    daily_ret = eq.pct_change().dropna()

    # Annualized volatility & Sharpe
    ann_vol = daily_ret.std() * np.sqrt(252) if not daily_ret.empty else np.nan
    metrics["annualized_volatility"] = ann_vol
    ann_return = daily_ret.mean() * 252 if not daily_ret.empty else np.nan
    sharpe = ann_return / ann_vol if ann_vol and not np.isnan(ann_vol) and ann_vol != 0 else np.nan
    metrics["sharpe_ratio"] = sharpe

    # Max drawdown
    rolling_max = eq.cummax()
    dd = (eq / rolling_max) - 1.0
    max_dd = dd.min() if not dd.empty else np.nan
    metrics["max_drawdown"] = max_dd

    # Trade stats
    if trades_df is not None and not trades_df.empty:
        wins = trades_df[trades_df["pnl"] > 0]
        losses = trades_df[trades_df["pnl"] <= 0]
        metrics["num_trades"] = len(trades_df)
        metrics["win_rate"] = len(wins) / len(trades_df) if len(trades_df) > 0 else np.nan
        metrics["avg_win"] = wins["return_pct"].mean() if not wins.empty else np.nan
        metrics["avg_loss"] = losses["return_pct"].mean() if not losses.empty else np.nan
    else:
        metrics["num_trades"] = 0
        metrics["win_rate"] = np.nan
        metrics["avg_win"] = np.nan
        metrics["avg_loss"] = np.nan

    return metrics


# ---------------------------
# Backtest Engine
# ---------------------------

def backtest_signals(
    signals: List[Signal],
    price_data: Dict[str, pd.DataFrame],
    fast_window: int,
    slow_window: int,
    atr_window: int,
) -> Tuple[pd.DataFrame, pd.DataFrame, List[TradeResult]]:
    """
    Simulate trades for all signals across tickers.
    - Executes exits at NEXT OPEN for (death cross / trailing stop / hard stop),
      and at CLOSE for max-hold rule (on the scheduled day).
    - Allows multiple concurrent positions across tickers. Only one open position per ticker.
    - Uses Adjusted Close for daily equity valuation; UNADJUSTED prices for execution.
    """

    # Index price_data keys to ensure all requested tickers present
    for s in signals:
        if s.ticker not in price_data:
            logger.warning(f"Signal for {s.ticker} but no price data; skipping.")
    signals = [s for s in signals if (s.ticker in price_data and s.entry_date is not None and s.entry_price is not None)]

    # Sort signals by entry_date (stable)
    signals.sort(key=lambda x: (x.entry_date, x.ticker))

    # Precompute death cross date sets per ticker for quick lookup
    death_cross_map: Dict[str, pd.DatetimeIndex] = {}
    for tkr, df in price_data.items():
        death_cross_map[tkr] = detect_death_cross_dates(df)

    # Master calendar (union of all trading days)
    all_dates = sorted(set().union(*[df.index.tolist() for df in price_data.values()]))
    master_calendar = pd.DatetimeIndex(all_dates)

    cash = float(INITIAL_CAPITAL)
    open_positions: Dict[str, Position] = {}
    trades: List[TradeResult] = []

    # Map entry schedule: date -> list of signals
    entries_by_date: Dict[pd.Timestamp, List[Signal]] = {}
    for s in signals:
        entries_by_date.setdefault(s.entry_date, []).append(s)

    # Equity curve records
    equity_rows = []

    # Helper for share sizing
    def size_shares(entry_px_exec: float, cash_available: float) -> Tuple[int, str]:
        note = ""
        if POSITION_SIZE_TYPE == "fixed_amount":
            alloc = min(POSITION_AMOUNT, cash_available)
        else:
            alloc = min(POSITION_PERCENT * cash_available, cash_available)
        # Account for commission in allocation
        alloc_minus_fees = max(0.0, alloc - COMMISSION_PER_TRADE)
        if entry_px_exec <= 0 or alloc_minus_fees <= 0:
            return 0, "Insufficient allocation after commission"
        shares_ideal = alloc_minus_fees / entry_px_exec
        shares = int(math.floor(shares_ideal))
        if shares <= 0:
            return 0, "Insufficient cash for 1 share after commission"
        if shares < shares_ideal:
            note = "Partial fill due to rounding/allocation"
        return shares, note

    # Iterate day by day
    for current_day in master_calendar:
        # 1) Execute any scheduled exits for NEXT OPEN (today)
        # We must process exits before entries to avoid immediate churn.
        to_close_now: List[Tuple[str, Position]] = []
        for tkr, pos in list(open_positions.items()):
            if pos.scheduled_exit_next_open and pos.scheduled_exit_next_open[0] == current_day:
                to_close_now.append((tkr, pos))

        for tkr, pos in to_close_now:
            df = price_data[tkr]
            if current_day not in df.index:
                logger.warning(f"{tkr} exit (next open) on {current_day.date()} skipped due to no data.")
                continue
            exit_open = df.at[current_day, "Open"]
            if pd.isna(exit_open) or exit_open <= 0:
                # Fallback to Close if needed
                exit_open = df.at[current_day, "Close"]
            exec_px = apply_slippage(exit_open, is_buy=False, slippage_pct=SLIPPAGE_PCT)
            # Close trade
            gross = exec_px * pos.shares
            cash += gross - COMMISSION_PER_TRADE
            days_held = (current_day - pos.entry_date).days
            pnl = (exec_px - pos.entry_price_exec) * pos.shares - COMMISSION_PER_TRADE  # includes commission on both sides
            ret_pct = (exec_px / pos.entry_price_exec) - 1.0
            reason = pos.scheduled_exit_next_open[1]
            trades.append(TradeResult(
                ticker=tkr,
                entry_date=pos.entry_date,
                entry_price=pos.entry_price_exec,
                exit_date=current_day,
                exit_price=exec_px,
                shares=pos.shares,
                pnl=pnl,
                return_pct=ret_pct,
                days_held=days_held,
                reason=reason
            ))
            del open_positions[tkr]

        # 2) Execute Entries scheduled TODAY at OPEN
        if current_day in entries_by_date:
            for s in entries_by_date[current_day]:
                tkr = s.ticker
                if tkr in open_positions:
                    logger.info(f"{tkr} already has an open position; skipping new entry on {current_day.date()}.")
                    continue
                df = price_data[tkr]
                if current_day not in df.index:
                    logger.warning(f"{tkr} no data for entry on {current_day.date()}; skipping.")
                    continue
                # Use today's open (fallback to close)
                raw_open = df.at[current_day, "Open"]
                if pd.isna(raw_open) or raw_open <= 0:
                    raw_open = df.at[current_day, "Close"]
                exec_px = apply_slippage(raw_open, is_buy=True, slippage_pct=SLIPPAGE_PCT)
                shares, sizing_note = size_shares(exec_px, cash)
                if shares <= 0:
                    logger.info(f"{tkr} entry skipped (insufficient cash) on {current_day.date()}. {sizing_note}")
                    continue

                # Deduct cash (include commission)
                cost = exec_px * shares + COMMISSION_PER_TRADE
                if cost > cash + 1e-9:
                    # Safety check: reduce shares to what cash allows
                    shares_can = int(math.floor((cash - COMMISSION_PER_TRADE) / exec_px))
                    if shares_can <= 0:
                        logger.info(f"{tkr} entry skipped (insufficient cash) on {current_day.date()}.")
                        continue
                    if shares_can < shares:
                        logger.info(f"{tkr} entry partially filled: requested {shares}, got {shares_can}.")
                        shares = shares_can
                        cost = exec_px * shares + COMMISSION_PER_TRADE

                cash -= cost

                # Initialize position
                highest_close = df.at[current_day, "Close"]
                trailing_stop = None
                hard_stop = None
                max_hold_exit = None

                if USE_TRAILING_STOP and not pd.isna(df.at[current_day, "ATR"]):
                    trailing_stop = float(max(np.nan, exec_px - TRAILING_ATR_MULTIPLIER * float(df.at[current_day, "ATR"])))
                if USE_HARD_STOP and HARD_STOP_PCT and HARD_STOP_PCT > 0:
                    hard_stop = float(exec_px * (1.0 - HARD_STOP_PCT))
                if MAX_HOLD_DAYS and MAX_HOLD_DAYS > 0:
                    # Exit at close on (entry_date + MAX_HOLD_DAYS)
                    # Find the corresponding calendar date that exists in the ticker's df
                    target_date = s.entry_date + timedelta(days=MAX_HOLD_DAYS)
                    # Snap to the next available trading day ON or AFTER target_date
                    dfi = df.index
                    idx = dfi.searchsorted(target_date)
                    if idx < len(dfi):
                        max_hold_exit = dfi[idx]
                    else:
                        max_hold_exit = None  # will never trigger if beyond data

                open_positions[tkr] = Position(
                    ticker=tkr,
                    entry_date=current_day,
                    entry_price_exec=exec_px,
                    raw_entry_price=float(raw_open),
                    shares=shares,
                    highest_close_since_entry=float(highest_close) if not pd.isna(highest_close) else float(exec_px),
                    trailing_stop=trailing_stop,
                    hard_stop=hard_stop,
                    max_hold_exit_date=max_hold_exit,
                )

        # 3) During the day: update stops & schedule future exits for NEXT OPEN if triggered
        for tkr, pos in list(open_positions.items()):
            df = price_data[tkr]
            if current_day not in df.index:
                continue

            # Update highest close since entry
            c_close = df.at[current_day, "Close"]
            if not pd.isna(c_close):
                pos.highest_close_since_entry = max(pos.highest_close_since_entry, float(c_close))

            # Update trailing stop (up only)
            if USE_TRAILING_STOP:
                atr = df.at[current_day, "ATR"] if "ATR" in df.columns else np.nan
                if not pd.isna(atr):
                    new_trail = pos.highest_close_since_entry - TRAILING_ATR_MULTIPLIER * float(atr)
                    if pos.trailing_stop is None:
                        pos.trailing_stop = float(new_trail)
                    else:
                        pos.trailing_stop = max(pos.trailing_stop, float(new_trail))

            # Check trigger conditions and schedule exit at NEXT OPEN
            today_low = df.at[current_day, "Low"]
            # Death cross today?
            if current_day in death_cross_map.get(tkr, pd.DatetimeIndex([])):
                next_day = first_next_trading_day(df, current_day)
                if next_day is not None:
                    if (pos.scheduled_exit_next_open is None) or (next_day < pos.scheduled_exit_next_open[0]):
                        pos.scheduled_exit_next_open = (next_day, "death_cross")

            # Trailing stop breach?
            if USE_TRAILING_STOP and pos.trailing_stop is not None and not pd.isna(today_low):
                if float(today_low) <= pos.trailing_stop:
                    next_day = first_next_trading_day(df, current_day)
                    if next_day is not None:
                        if (pos.scheduled_exit_next_open is None) or (next_day < pos.scheduled_exit_next_open[0]):
                            pos.scheduled_exit_next_open = (next_day, "trailing_stop")

            # Hard stop breach?
            if USE_HARD_STOP and pos.hard_stop is not None and not pd.isna(today_low):
                if float(today_low) <= pos.hard_stop:
                    next_day = first_next_trading_day(df, current_day)
                    if next_day is not None:
                        if (pos.scheduled_exit_next_open is None) or (next_day < pos.scheduled_exit_next_open[0]):
                            pos.scheduled_exit_next_open = (next_day, "hard_stop")

            # Max hold at CLOSE today?
            if pos.max_hold_exit_date is not None and current_day == pos.max_hold_exit_date:
                # Schedule for at-close today
                pos.scheduled_exit_at_close = (current_day, "max_hold_days")

        # 4) End of day: execute exits scheduled AT CLOSE today
        to_close_eod: List[Tuple[str, Position]] = [
            (tkr, pos) for tkr, pos in open_positions.items()
            if pos.scheduled_exit_at_close is not None and pos.scheduled_exit_at_close[0] == current_day
        ]
        for tkr, pos in to_close_eod:
            df = price_data[tkr]
            if current_day not in df.index:
                logger.warning(f"{tkr} scheduled close exit on {current_day.date()} missing data; skipping.")
                continue
            close_px = df.at[current_day, "Close"]
            if pd.isna(close_px) or close_px <= 0:
                # fallback to open
                close_px = df.at[current_day, "Open"]
            exec_px = apply_slippage(close_px, is_buy=False, slippage_pct=SLIPPAGE_PCT)
            # Close trade
            gross = exec_px * pos.shares
            cash += gross - COMMISSION_PER_TRADE
            days_held = (current_day - pos.entry_date).days
            pnl = (exec_px - pos.entry_price_exec) * pos.shares - COMMISSION_PER_TRADE
            ret_pct = (exec_px / pos.entry_price_exec) - 1.0
            reason = pos.scheduled_exit_at_close[1]
            trades.append(TradeResult(
                ticker=tkr,
                entry_date=pos.entry_date,
                entry_price=pos.entry_price_exec,
                exit_date=current_day,
                exit_price=exec_px,
                shares=pos.shares,
                pnl=pnl,
                return_pct=ret_pct,
                days_held=days_held,
                reason=reason
            ))
            del open_positions[tkr]

        # 5) Record equity curve for the day (valuation using Adj Close)
        positions_value = 0.0
        for tkr, pos in open_positions.items():
            df = price_data[tkr]
            if current_day in df.index:
                # Use Adjusted Close for portfolio valuation
                adj_px = df.at[current_day, "Adj Close"]
                if pd.isna(adj_px) or adj_px <= 0:
                    # fallback to Close if Adj Close unavailable
                    adj_px = df.at[current_day, "Close"]
                if not pd.isna(adj_px) and adj_px > 0:
                    positions_value += float(adj_px) * pos.shares
            else:
                # No data today; carry previous valuation (implicitly zero for that ticker today)
                pass

        portfolio_value = cash + positions_value
        equity_rows.append({
            "date": current_day,
            "portfolio_value": portfolio_value,
            "cash": cash,
            "positions_value": positions_value
        })

    equity_df = pd.DataFrame(equity_rows)
    trade_rows = []
    for tr in trades:
        trade_rows.append({
            "ticker": tr.ticker,
            "entry_date": tr.entry_date.strftime("%Y-%m-%d"),
            "entry_price": round(tr.entry_price, 6),
            "exit_date": tr.exit_date.strftime("%Y-%m-%d"),
            "exit_price": round(tr.exit_price, 6),
            "shares": tr.shares,
            "pnl": round(tr.pnl, 6),
            "return_pct": round(tr.return_pct * 100.0, 4),
            "days_held": tr.days_held,
            "reason": tr.reason
        })
    trades_df = pd.DataFrame(trade_rows)

    return equity_df, trades_df, trades


# ---------------------------
# Plotting Helpers
# ---------------------------

def plot_equity_curve(equity_df: pd.DataFrame, out_path: str):
    if equity_df is None or equity_df.empty:
        logger.warning("Equity curve is empty; skipping plot.")
        return
    plt.figure(figsize=(10, 5))
    plt.plot(equity_df["date"], equity_df["portfolio_value"])
    plt.title("Equity Curve")
    plt.xlabel("Date")
    plt.ylabel("Portfolio Value")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    logger.info(f"Saved equity curve plot to {out_path}.")


def plot_trade_returns_hist(trades_df: pd.DataFrame, out_path: str):
    if trades_df is None or trades_df.empty:
        logger.warning("No trades; skipping returns histogram.")
        return
    plt.figure(figsize=(8, 5))
    plt.hist(trades_df["return_pct"].astype(float), bins=30)
    plt.title("Trade Returns (%)")
    plt.xlabel("Return (%)")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    logger.info(f"Saved trade returns histogram to {out_path}.")


# ---------------------------
# Main Pipeline
# ---------------------------

def run_pipeline():
    # 1) Fetch data
    price_data: Dict[str, pd.DataFrame] = {}
    for t in TICKERS:
        df = fetch_data(t, START_DATE, END_DATE, interval=TIMEFRAME)
        if df is None or df.empty:
            continue
        df = compute_indicators(df, FAST_WINDOW, SLOW_WINDOW, ATR_WINDOW)
        price_data[t] = df

    if not price_data:
        logger.error("No price data fetched. Exiting.")
        return

    # 2) Detect signals per ticker
    all_signals: List[Signal] = []
    for t, df in price_data.items():
        sigs = detect_golden_crosses(df, t)
        all_signals.extend(sigs)

    # 3) Save signals CSV
    save_signals_csv(all_signals, OUTPUT_SIGNALS_CSV)

    # 4) Backtest signals
    equity_df, trades_df, _ = backtest_signals(
        signals=all_signals,
        price_data=price_data,
        fast_window=FAST_WINDOW,
        slow_window=SLOW_WINDOW,
        atr_window=ATR_WINDOW,
    )

    # 5) Save backtest results and equity curve
    if trades_df is not None and not trades_df.empty:
        trades_df.to_csv(OUTPUT_BACKTEST_CSV, index=False)
        logger.info(f"Saved backtest results to {OUTPUT_BACKTEST_CSV} ({len(trades_df)} trades).")
    else:
        logger.info("No completed trades to save.")

    if equity_df is not None and not equity_df.empty:
        # ensure date column is datetime
        equity_df["date"] = pd.to_datetime(equity_df["date"])
        equity_df.to_csv(OUTPUT_EQUITY_CURVE_CSV, index=False)
        logger.info(f"Saved equity curve to {OUTPUT_EQUITY_CURVE_CSV} ({len(equity_df)} rows).")

        # 6) Metrics
        metrics = calculate_metrics(trades_df, equity_df)
        logger.info("==== Summary Metrics ====")
        logger.info(f"Total Return: {metrics.get('total_return', np.nan):.4f}")
        logger.info(f"CAGR: {metrics.get('cagr', np.nan):.4f}")
        logger.info(f"Annualized Volatility: {metrics.get('annualized_volatility', np.nan):.4f}")
        logger.info(f"Sharpe Ratio: {metrics.get('sharpe_ratio', np.nan):.4f}")
        logger.info(f"Max Drawdown: {metrics.get('max_drawdown', np.nan):.4f}")
        logger.info(f"# Trades: {metrics.get('num_trades', 0)}")
        logger.info(f"Win Rate: {metrics.get('win_rate', np.nan):.4f}")
        logger.info(f"Avg Win (%): {metrics.get('avg_win', np.nan):.4f}")
        logger.info(f"Avg Loss (%): {metrics.get('avg_loss', np.nan):.4f}")

        # 7) Plots
        plot_equity_curve(equity_df, "output/equity_curve.png")
        if trades_df is not None and not trades_df.empty:
            plot_trade_returns_hist(trades_df, "output/trade_returns_hist.png")


if __name__ == "__main__":
    run_pipeline()


14:13:57 | INFO | Saved signals to output/goldencross_signals.csv (1364 rows).
14:13:58 | INFO | GILLETTE.NS entry skipped (insufficient cash) on 2020-10-26. Insufficient cash for 1 share after commission
14:13:58 | INFO | KOTAKBANK.NS entry skipped (insufficient cash) on 2020-11-06. Insufficient cash for 1 share after commission
14:13:58 | INFO | PAGEIND.NS entry skipped (insufficient cash) on 2020-11-06. Insufficient cash for 1 share after commission
14:13:58 | INFO | MRF.NS entry skipped (insufficient cash) on 2020-11-18. Insufficient cash for 1 share after commission
14:13:58 | INFO | SHREECEM.NS entry skipped (insufficient cash) on 2020-11-23. Insufficient cash for 1 share after commission
14:13:58 | INFO | BATAINDIA.NS entry skipped (insufficient cash) on 2020-11-26. Insufficient cash for 1 share after commission
14:13:58 | INFO | 3MINDIA.NS entry skipped (insufficient cash) on 2020-12-01. Insufficient cash for 1 share after commission
14:13:58 | INFO | BAJAJHLDNG.NS entry skippe